In [139]:
import pandas as pd 
import numpy as np 
import seaborn as sns 
sns.set(style='dark', palette='muted', color_codes=True)
import matplotlib.pyplot as plt 
plt.style.use('ggplot')
import plotly.express as px 

In [140]:
from sklearn.impute import SimpleImputer 
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder, OneHotEncoder, MinMaxScaler, StandardScaler
from sklearn.compose import ColumnTransformer 
from sklearn.decomposition import PCA
from sklearn.metrics import r2_score, mean_squared_error, silhouette_score, confusion_matrix, accuracy_score 
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.pipeline import Pipeline 
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.cluster import KMeans

In [141]:
data = pd.read_csv(r'c:\Github\Sol\the_data\data-lab-3-insurance.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1338 non-null   int64  
 1   sex       1338 non-null   object 
 2   bmi       1338 non-null   float64
 3   children  1338 non-null   int64  
 4   smoker    1338 non-null   object 
 5   region    1338 non-null   object 
 6   charges   1338 non-null   float64
dtypes: float64(2), int64(2), object(3)
memory usage: 73.3+ KB


In [142]:
mis_val = data.isna().sum() * 100/len(data)
mis_val.sort_values(ascending=False)

age         0.0
sex         0.0
bmi         0.0
children    0.0
smoker      0.0
region      0.0
charges     0.0
dtype: float64

In [143]:
data.duplicated().sum()

data.drop_duplicates(subset=None, inplace=True, keep='first', ignore_index=False)
data.duplicated().sum()

0

In [144]:
le = LabelEncoder()

In [145]:
orig_sex = data.sex 

data.sex = le.fit_transform(data.sex)
data.region = le.fit_transform(data.region)
data.smoker = le.fit_transform(data.smoker)

le_sex = data.sex

In [146]:
orig_sex_dic = np.unique(orig_sex)
le_sex_dic = np.unique(le_sex)

sex_dic = dict(zip(orig_sex_dic,le_sex_dic))
sex_dic

{'female': 0, 'male': 1}

In [147]:
x = data.drop(columns=['charges'])
y = data.charges
# y

In [148]:
scaler = StandardScaler()

x = pd.DataFrame(scaler.fit_transform(x))
y = scaler.fit_transform(np.array(y).reshape(-1,1))


In [149]:
x_train,x_test,y_train,y_test = train_test_split(x,y,random_state=42,test_size=0.3)

In [150]:

linreg = LinearRegression()

linreg.fit(x_train,y_train)

y_true=y_test

y_pred = linreg.predict(x_test)

scr_r2 = r2_score(y_true,y_pred)

mod_per = pd.DataFrame( 
    [
    'LinearRegression', scr_r2
    ]
).transpose()

mod_per.columns = ['Method', 'Test R2']

mod_per

,Method,Test R2
0,LinearRegression,0.772465


In [151]:
ranfor = RandomForestRegressor(n_estimators=100, n_jobs=-1, random_state=42, criterion='squared_error')

ranfor.fit(x_train,y_train)

y_pred = ranfor.predict(x_test)

mse_scr = mean_squared_error(y_true,y_pred).round(3)

perf_mod = pd.DataFrame( 
    [ 
        'RandomForestRegressor', mse_scr
    ]
).transpose()

perf_mod.columns = ['Method', 'RanFor MSE']

perf_mod

C:\Users\amunv\AppData\Local\Temp\ipykernel_23580\3949086132.py:3: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



,Method,RanFor MSE
0,RandomForestRegressor,0.16
